In [35]:
import pandas as pd
import numpy as np
import csv
import sklearn
from sklearn.decomposition import TruncatedSVD

In [36]:
from surprise import SVD

In [37]:
columns = ['userid', 'movieId', 'rating', 'timestamp']
ratings = pd.read_excel('ratings.xlsx')


In [38]:

################################################################################
columns = ['movieId', 'movie title', 'release date', 'video release date', 'IMDb URL', 'unknown', 'Action', 'Adventure',
           'Animation', 'Childrens', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror',
           'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']
           


In [39]:
movies = pd.read_excel('movies.xlsx')
new_movies = movies['genres'].str.get_dummies(sep='|')
movies.drop('genres',inplace=True,axis=1)
movies=pd.concat([movies, new_movies], axis=1)
movies.head()

,movieId,title,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,Documentary,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1.0,Toy Story (1995),0,0,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2.0,Jumanji (1995),0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3.0,Grumpier Old Men (1995),0,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
3,4.0,Waiting to Exhale (1995),0,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
4,5.0,Father of the Bride Part II (1995),0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [40]:

ratings = pd.merge(movies, ratings, on='movieId')
ratings.drop('(no genres listed)',inplace=True,axis=1)
ratings.head()

,movieId,title,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,...,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,userId,rating,timestamp
0,1.0,Toy Story (1995),0,1,1,1,1,0,0,0,...,0,0,0,0,0,0,0,1.0,4.0,9.649827e+08
1,1.0,Toy Story (1995),0,1,1,1,1,0,0,0,...,0,0,0,0,0,0,0,5.0,4.0,8.474350e+08
2,1.0,Toy Story (1995),0,1,1,1,1,0,0,0,...,0,0,0,0,0,0,0,7.0,4.5,1.106636e+09
3,1.0,Toy Story (1995),0,1,1,1,1,0,0,0,...,0,0,0,0,0,0,0,15.0,2.5,1.510578e+09
4,1.0,Toy Story (1995),0,1,1,1,1,0,0,0,...,0,0,0,0,0,0,0,17.0,4.5,1.305696e+09


In [41]:
ratings.columns

Index(['movieId', 'title', 'Action', 'Adventure', 'Animation', 'Children',
       'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir',
       'Horror', 'IMAX', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller',
       'War', 'Western', 'userId', 'rating', 'timestamp'],
      dtype='object')

In [42]:
ratings['userId']

0           1.0
1           5.0
2           7.0
3          15.0
4          17.0
          ...  
100831    184.0
100832    184.0
100833    184.0
100834    184.0
100835    331.0
Name: userId, Length: 100836, dtype: float64

In [43]:
rating_crosstab = ratings.pivot_table(
    values='rating', index='userId', columns='title', fill_value=0)
print(rating_crosstab.head())
X=rating_crosstab.T

title   (500) Days of Summer (2009)  *batteries not included (1987)  \
userId                                                                
1.0                             0.0                             0.0   
2.0                             0.0                             0.0   
3.0                             0.0                             0.0   
4.0                             0.0                             0.0   
5.0                             0.0                             0.0   

title   ...All the Marbles (1981)  ...And Justice for All (1979)  \
userId                                                             
1.0                             0                            0.0   
2.0                             0                            0.0   
3.0                             0                            0.0   
4.0                             0                            0.0   
5.0                             0                            0.0   

title   00 Schneider - Ja

In [44]:

SVD = TruncatedSVD(n_components=12, random_state=5)
resultant_matrix = SVD.fit_transform(X)
print(resultant_matrix.shape)
corr_matrix=np.corrcoef(resultant_matrix)
print(corr_matrix.shape)

(9719, 12)
(9719, 9719)


In [45]:
from surprise import Reader, Dataset
reader = Reader()

In [46]:
ratings[['userId', 'movieId', 'rating']]

,userId,movieId,rating
0,1.0,1.0,4.0
1,5.0,1.0,4.0
2,7.0,1.0,4.5
3,15.0,1.0,2.5
4,17.0,1.0,4.5
...,...,...,...
100831,184.0,193581.0,4.0
100832,184.0,193583.0,3.5
100833,184.0,193585.0,3.5
100834,184.0,193587.0,3.5


In [54]:
from surprise import Reader, Dataset, SVD
reader = Reader()
ratings = pd.read_csv('ratings_small.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [55]:
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

In [56]:
from surprise.model_selection import KFold
kf = KFold(n_splits=5)
kf.split(data)

<generator object KFold.split at 0x000001A500B7D580>

In [59]:
svd = SVD()
from surprise.model_selection import cross_validate
scores=cross_validate(svd, data, measures=['RMSE', 'MAE'],cv=5,verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8940  0.8974  0.8961  0.8932  0.8946  0.8951  0.0015  
MAE (testset)     0.6890  0.6914  0.6899  0.6885  0.6914  0.6900  0.0012  
Fit time          3.95    3.92    3.92    3.91    3.86    3.91    0.03    
Test time         0.12    0.11    0.16    0.10    0.11    0.12    0.02    


In [85]:
ff= pd.DataFrame(scores)
ff

,test_rmse,test_mae,fit_time,test_time
0,0.894029,0.688971,3.953204,0.117162
1,0.897410,0.691433,3.923239,0.107969
2,0.896051,0.689899,3.915723,0.160571
3,0.893222,0.688529,3.911572,0.101729
4,0.894627,0.691387,3.857021,0.108709


In [87]:
ff['test_rmse']=ff['test_rmse']*100
ff['test_mae']=ff['test_mae']*100
ff

,test_rmse,test_mae,fit_time,test_time
0,89.402941,68.897066,3.953204,0.117162
1,89.740987,69.143283,3.923239,0.107969
2,89.605145,68.989946,3.915723,0.160571
3,89.322222,68.852912,3.911572,0.101729
4,89.462660,69.138721,3.857021,0.108709


In [69]:
cols=[]
values=[]
for val in scores:
    cols.append(val)
    values.append(scores[val])

In [ ]:
trainset = data.build_full_trainset()
svd.fit(trainset)

In [ ]:
col_idx = rating_crosstab.columns.get_loc("Jurassic Park (1993)")

corr_specific = corr_matrix[col_idx]

pd.DataFrame({'corr_specific': corr_specific, 'Movies': rating_crosstab.columns})\
    .sort_values('corr_specific', ascending=False)\
    .head(10)
#####################################################################

,corr_specific,Movies
4656,1.000000,Jurassic Park (1993)
829,0.987646,Batman (1989)
8947,0.975182,True Lies (1994)
7908,0.972973,Speed (1994)
4331,0.971110,Independence Day (a.k.a. ID4) (1996)
5705,0.964891,Mission: Impossible (1996)
8359,0.963661,Terminator 2: Judgment Day (1991)
3522,0.963176,GoldenEye (1995)
5482,0.962586,"Mask, The (1994)"
3272,0.960650,"Fugitive, The (1993)"


In [ ]:
col_idx = rating_crosstab.columns.get_loc("Balto (1995)")

corr_specific = corr_matrix[col_idx]

pd.DataFrame({'corr_specific': corr_specific, 'Movies': rating_crosstab.columns})\
    .sort_values('corr_specific', ascending=False)\
    .head(10)


,corr_specific,Movies
778,1.000000,Balto (1995)
6906,0.911048,Quest for Camelot (1998)
4125,0.880577,"Hunchback of Notre Dame, The (1996)"
292,0.876424,Aladdin and the King of Thieves (1996)
7801,0.862222,Snow White and the Seven Dwarfs (1937)
312,0.843662,Alice (2009)
2890,0.843662,FairyTale: A True Story (1997)
506,0.843662,Animal Farm (1954)
1781,0.843662,Cirque du Freak: The Vampire's Assistant (2009)
2224,0.843662,Dead Like Me: Life After Death (2009)


In [ ]:
col_idx = rating_crosstab.columns.get_loc("Stalag 17 (1953)")

corr_specific = corr_matrix[col_idx]

pd.DataFrame({'corr_specific': corr_specific, 'Movies': rating_crosstab.columns})\
    .sort_values('corr_specific', ascending=False)\
    .head(10)


,corr_specific,Movies
7964,1.000000,Stalag 17 (1953)
8757,0.955440,"Time Machine, The (1960)"
6579,0.932743,Peter's Friends (1992)
6190,0.922514,Nothing to Lose (1997)
9455,0.918555,"Widow of St. Pierre, The (Veuve de Saint-Pierr..."
184,0.918338,About Schmidt (2002)
3140,0.911314,Forbidden Planet (1956)
2217,0.910526,Dead Again (1991)
1365,0.910520,"Bridge on the River Kwai, The (1957)"
4690,0.907771,"Karate Kid, Part III, The (1989)"


In [ ]:
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)


NameError: name 'Dataset' is not defined